In [32]:
import duckdb
import pandas as pd
import numpy as np
from itables import init_notebook_mode, show

In [11]:
import pandas as pd
import re
from collections import Counter

# --- User parameters ---
parquet_path = '../data/blip_captioned_images.parquet'  # Update this to your actual parquet file path
caption_col = 'caption'            # Update if your caption column has a different name

# --- Intensity lexicon definition ---
INTENSITY_LEXICON = {
    5: ["obliterated", "annihilated", "wiped out", "pulverized", "eradicated"],
    4: ["razed", "flattened", "leveled", "broken", "torn apart"],
    3: ["demolished", "shattered", "in ruins", "wrecked", "ruined"],
    2: ["hit", "struck", "battered", "impacted", "affected"],
    1: ["scratched", "marred", "nicked", "marked", "touched"],
}

def compile_patterns(lexicon):
    """Compile regex patterns for each term to match whole words, case‐insensitive."""
    patterns = {}
    for weight, terms in lexicon.items():
        patterns[weight] = [
            (term, re.compile(r'\b' + re.escape(term) + r'\b', flags=re.IGNORECASE))
            for term in terms
        ]
    return patterns

def count_terms_in_caption(caption, patterns):
    """Count occurrences of each term in a single caption."""
    counts = Counter()
    text = str(caption)
    for weight, term_patterns in patterns.items():
        for term, pat in term_patterns:
            counts[weight] += len(pat.findall(text))
    return counts

def aggregate_counts(df, caption_col):
    """Aggregate counts per intensity category across all captions."""
    patterns = compile_patterns(INTENSITY_LEXICON)
    total_counts = Counter()
    for caption in df[caption_col].fillna('').astype(str):
        total_counts.update(count_terms_in_caption(caption, patterns))
    return {w: total_counts[w] for w in sorted(INTENSITY_LEXICON.keys(), reverse=True)}

# --- Load captions and compute counts ---
df = pd.read_parquet(parquet_path)
counts_dict = aggregate_counts(df, caption_col)

# --- Prepare results DataFrame ---
results_df = pd.DataFrame([
    {
        "Intensity": weight,
        "Terms": ", ".join(INTENSITY_LEXICON[weight]),
        "Count": counts_dict[weight]
    }
    for weight in sorted(INTENSITY_LEXICON.keys(), reverse=True)
])

# --- Display results ---
print("Destruction Framing Counts", results_df)


Destruction Framing Counts    Intensity                                              Terms  Count
0          5  obliterated, annihilated, wiped out, pulverize...     32
1          4      razed, flattened, leveled, broken, torn apart    185
2          3   demolished, shattered, in ruins, wrecked, ruined    290
3          2          hit, struck, battered, impacted, affected   2377
4          1         scratched, marred, nicked, marked, touched    114


In [13]:
import pandas as pd
import re
from collections import Counter

# Path to your Parquet file
parquet_path = '../data/filtered_articles_date_cleaned.parquet'

# Columns to combine (handle nulls safely)
text_cols = ['title', 'description', 'text']

# 1. Load the Parquet (requires pyarrow or fastparquet)
df = pd.read_parquet(parquet_path)

# 2. Ensure all text columns exist and fill nulls
for col in text_cols:
    if col not in df.columns:
        df[col] = ''
    else:
        df[col] = df[col].fillna('')

# 3. Combine into a single “full_text” field
df['full_text'] = df['title'] + ' ' + df['description'] + ' ' + df['text']

# 4. Define your intensity lexicon
INTENSITY_LEXICON = {
    5: ["obliterated", "annihilated", "wiped out", "pulverized", "eradicated"],
    4: ["razed", "flattened", "leveled", "broken", "torn apart"],
    3: ["demolished", "shattered", "in ruins", "wrecked", "ruined"],
    2: ["hit", "struck", "battered", "impacted", "affected"],
    1: ["scratched", "marred", "nicked", "marked", "touched"],
}

# 5. Compile regex patterns for each term
def compile_patterns(lexicon):
    return {
        weight: [
            (term, re.compile(r'\b' + re.escape(term) + r'\b', flags=re.IGNORECASE))
            for term in terms
        ]
        for weight, terms in lexicon.items()
    }

patterns = compile_patterns(INTENSITY_LEXICON)

# 6. Function to count terms in a text
def count_terms(text, patterns):
    counts = Counter()
    s = str(text)
    for weight, pats in patterns.items():
        for _, pat in pats:
            counts[weight] += len(pat.findall(s))
    return counts

# 7. Initialize counters for “Gaza” vs “Israel”
gaza_counts = Counter()
israel_counts = Counter()

# 8. Iterate over articles and update counts
for txt in df['full_text']:
    lc = txt.lower()
    c = count_terms(txt, patterns)
    if 'gaza' in lc:
        gaza_counts.update(c)
    if 'israel' in lc:
        israel_counts.update(c)

# 9. Build results DataFrame
results = [
    {
        "Intensity": w,
        "Terms": ", ".join(INTENSITY_LEXICON[w]),
        "Gaza_Count": gaza_counts[w],
        "Israel_Count": israel_counts[w],
    }
    for w in sorted(INTENSITY_LEXICON.keys(), reverse=True)
]
results_df = pd.DataFrame(results)

# 10. Display in notebook
print("Destruction Framing by Side (Articles)", results_df)


Destruction Framing by Side (Articles)    Intensity                                              Terms  Gaza_Count  \
0          5  obliterated, annihilated, wiped out, pulverize...        1148   
1          4      razed, flattened, leveled, broken, torn apart        3655   
2          3   demolished, shattered, in ruins, wrecked, ruined        2959   
3          2          hit, struck, battered, impacted, affected       25665   
4          1         scratched, marred, nicked, marked, touched        2748   

   Israel_Count  
0          1206  
1          3833  
2          3061  
3         27348  
4          2944  


In [15]:
import os
import pandas as pd
import re
from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS


# --- Load data with fallback to CSV if Parquet engine is unavailable ---
parquet_path = '../data/filtered_articles_date_cleaned.parquet'
csv_path = '../data/filtered_articles_date_cleaned.csv'

if os.path.exists(parquet_path):
    try:
        df = pd.read_parquet(parquet_path)
    except Exception:
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
        else:
            raise RuntimeError(
                "Unable to read Parquet (no engine installed) and no CSV fallback found. "
                "Please convert to CSV named 'filtered_articles_date_cleaned.csv'."
            )
elif os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
else:
    raise FileNotFoundError("Neither Parquet nor CSV file found.")

# --- Ensure text columns exist and fill nulls ---
for col in ['title', 'description', 'text']:
    df[col] = df.get(col, '').fillna('')

# --- Combine into a single text field ---
df['full_text'] = df['title'] + ' ' + df['description'] + ' ' + df['text']

# --- Tokenize and remove English stop words ---
stopwords = set(ENGLISH_STOP_WORDS)
word_counter = Counter()
pattern = re.compile(r'\b\w+\b')

for txt in df['full_text'].astype(str):
    tokens = pattern.findall(txt.lower())
    filtered = [t for t in tokens if t not in stopwords and len(t) > 1]
    word_counter.update(filtered)

# --- Build and save frequency DataFrame ---
freq_df = pd.DataFrame(word_counter.items(), columns=['word', 'count'])
freq_df = freq_df.sort_values('count', ascending=False).reset_index(drop=True)
freq_df.to_csv('word_frequency.csv', index=False)

# --- Display top 20 words ---
print("Top 20 Word Frequencies", freq_df.head(20))


Top 20 Word Frequencies             word   count
0         israel  439745
1           said  357344
2           gaza  344263
3          hamas  281608
4        israeli  259536
5            war  143891
6         people  127292
7    palestinian  104071
8         killed  102174
9       military   98413
10      hostages   85317
11        attack   78969
12     netanyahu   73745
13  palestinians   69355
14           aid   66376
15      minister   65992
16          iran   63504
17         biden   62707
18     hezbollah   60834
19     president   59645


In [53]:
import os
import pandas as pd
import re
from tqdm import tqdm

# --- Load data ---
parquet_path = '../data/filtered_articles_date_cleaned.parquet'
csv_path     = '../data/filtered_articles_date_cleaned.csv'

if os.path.exists(parquet_path):
    try:
        df = pd.read_parquet(parquet_path)
        df=df[df["source_domain"]=="bbc.com"]
    except:
        df = pd.read_csv(csv_path)
else:
    df = pd.read_csv(csv_path)

# --- Clean necessary columns ---
for col in ['id', 'source_domain', 'title', 'description', 'text']:
    if col not in df.columns:
        df[col] = ''
df.fillna('', inplace=True)

# --- Compile stem regex ---
stems = ['rubble', 'destr', 'ruin', 'devast', 'demolish', 'crater', 'bomb']
destruction_pattern = re.compile(r'\b\w*(?:' + '|'.join(stems) + r')\w*\b', re.IGNORECASE)

# --- Extract matched sentences ---
records = []

for row in tqdm(df.itertuples(index=False), total=len(df), desc="Scanning articles"):
    article_id = row.id
    source     = row.source_domain
    full_text  = f"{row.title} {row.description} {row.text}"
    sentences  = re.split(r'(?<=[\.\!\?])\s+', full_text)

    for sent in sentences:
        for match in destruction_pattern.finditer(sent):
            word = match.group(0).lower()
            records.append({
                'article_id': article_id,
                'source_domain': source,
                'sentence': sent.strip(),
                'keyword_found': word,
                'title': row.title.strip(),
                'description': row.description.strip()
            })

# --- Create and save final DataFrame ---
out_df = pd.DataFrame(records)
out_df.to_parquet("../data/destruction_sentences_flat.parquet", index=False)

print(f"✅ Saved {len(out_df):,} destruction-tagged sentences across {len(df):,} articles.")


Scanning articles: 100%|██████████| 2376/2376 [00:01<00:00, 1719.67it/s]


✅ Saved 3,766 destruction-tagged sentences across 2,376 articles.


In [59]:
import pandas as pd
from transformers import pipeline

#
df = out_df

# 2) QA pipeline on GPU
qa = pipeline(
    "question-answering",
    model="distilbert-base-cased-distilled-squad",
    tokenizer="distilbert-base-cased-distilled-squad",
    device=0,
)

questions = {
    "agent":    "Who is responsible for the destruction?",
    "location": "What was destroyed?"
}

meta_inputs = []
qa_inputs   = []

for _, row in df.iterrows():
    for qkey, qtext in questions.items():
        meta_inputs.append({
            "article_id":    row.article_id,
            "source_domain": row.source_domain,
            "sentence":       row.sentence,
            "question_key":  qkey
        })
        qa_inputs.append({
            "question": qtext,
            "context":  row.sentence+ " " + row.title + " " + row.description,
        })

# 3) Run QA in batches
qa_results = qa(qa_inputs, batch_size=32)

# 4) Combine metadata + QA outputs
records = []
for meta, out in zip(meta_inputs, qa_results):
    rec = {
        "article_id":    meta["article_id"],
        "source_domain": meta["source_domain"],
        "sentence":       meta["sentence"],
        meta["question_key"]: out["answer"]
    }
    records.append(rec)

# 5) Pivot wide, including caption
df_long = pd.DataFrame(records)
df_wide = (
    df_long
    .groupby(
        ["article_id","source_domain","sentence"], 
        as_index=False
    )
    .agg({"agent":"first","location":"first"})
)

df_wide.head()


Device set to use cuda:0
/mnt/raid1/MSCs/AY2425/skaul/miniconda3/envs/bias-analysis/lib/python3.10/site-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


,article_id,source_domain,sentence,agent,location
0,00346eee-c87d-409c-abc1-d1e83d1d9d8d,bbc.com,President Joe Biden said he and first lady Jil...,President Joe Biden said he and first lady Jil...,Chen's passing
1,006a6cf1-ee91-4aa2-808d-8b0e8f3f0d2a,bbc.com,Intensive Israeli bombardment has led to the c...,Israeli,waste water management infrastructure
2,006a6cf1-ee91-4aa2-808d-8b0e8f3f0d2a,bbc.com,The destruction of Gaza's health care system w...,Gaza's health care system,health care system
3,006bf4c6-38e5-4199-ad4f-af9f0bf9aea1,bbc.com,"""The situation at Al-Awda Hospital is particul...",Children,one of the buildings
4,006bf4c6-38e5-4199-ad4f-af9f0bf9aea1,bbc.com,A lack of food resulted in the deaths of 10 ch...,hospital buildings,hospital buildings


In [60]:
show(df_wide)

Loading ITables v2.4.4 from the internet... (need help?)


In [58]:
# Filter the df_wide dataframe where agent or location contains 'israel' or 'gaza' (case insensitive)
filtered_df = df_wide[
    (df_wide['agent'].str.contains('israel|gaza', case=False, na=False)) |
    (df_wide['location'].str.contains('israel|gaza', case=False, na=False))
]

print(f"Count of records where agent or location contains 'israel' or 'gaza': {len(filtered_df)}")
show(filtered_df)

Count of records where agent or location contains 'israel' or 'gaza': 2210


Loading ITables v2.4.4 from the internet... (need help?)
